In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

OUTPUT_DIR = "../data/others/model_output"

folder_paths = [OUTPUT_DIR]

# Create the output directories if they doesn't exist
for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [2]:
import albumentations as A
from plot import get_bounding_boxes, create_bboxes_image
import cv2
import numpy as np
from layers import AMF_GD_YOLOv8
import matplotlib.pyplot as plt
from time import time
import random
import torch
import torch.nn as nn

In [3]:
print(torch.cuda.is_available())

True


### Run the YOLOv8 model

In [4]:
from ultralytics.models.yolo import YOLO

# Load a pretrained YOLO model
model = YOLO("yolov8n.pt")

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg", save=True)

100%|██████████| 6.23M/6.23M [00:00<00:00, 96.2MB/s]


100%|██████████| 476k/476k [00:00<00:00, 41.8MB/s]


image 1/1 /home/alexandre/Documents/tree-segmentation/src/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 47.1ms
Speed: 4.1ms preprocess, 47.1ms inference, 119.9ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict


### Use the AMF GD YOLOv8 model

In [5]:
class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}
class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, scale="s", class_names=class_names)

rgb_image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.tif"
chm_image_path = (
    "../data/CHM_cropped_0p24/122000_484000/122000_484000_1_1_filtered_chm.tif"
)
annotations_path = "../data/annotations/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.json"

gt_bboxes, gt_classes = get_bounding_boxes(annotations_path)

output_name = "Model_output_test.jpg"
output_path = os.path.join(OUTPUT_DIR, output_name)

model.train()
output = model(rgb_image_path, chm_image_path, image_save_path=output_path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/annotations_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.json'

In [22]:
# from ultralytics.models.utils.loss import DETRLoss
# from layers import xywh2xyxy

# nc = 4
# loss_func = DETRLoss(nc=nc)

# # output[0][0][0, :, 0] = torch.Tensor([107.3209, 151.2296, 161.4145, 207.6499, 0.9837, 0.5472, 0.3150, 0.5183])

# pred_bboxes = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, :4]
# pred_scores = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, 4:]
# batch = {
#     "cls": torch.Tensor([class_indices[cls] for cls in classes]).to(dtype=torch.int64),
#     "bboxes": xywh2xyxy(torch.Tensor(bboxes)),
#     "gt_groups": [len(classes)],
# }

# prefect_pred_bboxes = batch["bboxes"].unsqueeze(0).unsqueeze(0)
# prefect_pred_scores = (
#     torch.where(
#         batch["cls"].unsqueeze(-1).repeat(1, nc)
#         == torch.arange(nc).unsqueeze(0).repeat(batch["cls"].shape[0], 1),
#         1.0,
#         -1.0,
#     )
#     .unsqueeze(0)
#     .unsqueeze(0)
# )

# loss = loss_func.forward(pred_bboxes, pred_scores, batch)
# print(f"{loss = }")

model.train()
for i in range(100):
    output = model(rgb_image_path, chm_image_path, image_save_path=output_path)
    loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
    print(loss)
    (loss["loss_class"] + loss["loss_bbox"]).backward()

{'loss_class': tensor(1.6705, grad_fn=<MulBackward0>), 'loss_bbox': tensor(19366.4082, grad_fn=<SqueezeBackward0>), 'loss_giou': tensor(2.9315, grad_fn=<SqueezeBackward0>), 'loss_class_aux': tensor(0.), 'loss_bbox_aux': tensor(0.), 'loss_giou_aux': tensor(0.)}
{'loss_class': tensor(1.6705, grad_fn=<MulBackward0>), 'loss_bbox': tensor(19366.4082, grad_fn=<SqueezeBackward0>), 'loss_giou': tensor(2.9315, grad_fn=<SqueezeBackward0>), 'loss_class_aux': tensor(0.), 'loss_bbox_aux': tensor(0.), 'loss_giou_aux': tensor(0.)}
{'loss_class': tensor(1.6705, grad_fn=<MulBackward0>), 'loss_bbox': tensor(19366.4082, grad_fn=<SqueezeBackward0>), 'loss_giou': tensor(2.9315, grad_fn=<SqueezeBackward0>), 'loss_class_aux': tensor(0.), 'loss_bbox_aux': tensor(0.), 'loss_giou_aux': tensor(0.)}
{'loss_class': tensor(1.6728, grad_fn=<MulBackward0>), 'loss_bbox': tensor(19366.4082, grad_fn=<SqueezeBackward0>), 'loss_giou': tensor(2.9315, grad_fn=<SqueezeBackward0>), 'loss_class_aux': tensor(0.), 'loss_bbox_aux

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 100

dataloader = ...
model = AMF_GD_YOLOv8(3, 1, scale="n", class_names=class_names)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
)

model.train()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        rgb_image = ...
        chm_image = ...

        optimizer.zero_grad()

        output = model(rgb_image, chm_image)
        loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
        total_loss = loss["loss_class"] + loss["loss_bbox"]
        total_loss.backward()

        optimizer.step()

    scheduler.step()
    print(f"Loss class: {loss["loss_class"]:.4f}, Loss bbox: {loss["loss_bbox"]:.4f}")

### How to use data augmentation with Albumentations

In [8]:
from typing import Sequence, Dict, Any


class Pad(A.PadIfNeeded):
    def __init__(
        self,
        added_height: int | None = 64,
        added_width: int | None = 64,
        pad_height_divisor: int | None = None,
        pad_width_divisor: int | None = None,
        position: A.PadIfNeeded.PositionType | str = A.PadIfNeeded.PositionType.CENTER,
        border_mode: int = cv2.BORDER_REFLECT_101,
        value: float | Sequence[float] | None = None,
        mask_value: float | Sequence[float] | None = None,
        always_apply: bool = False,
        p: float = 1,
    ):
        super().__init__(
            0,
            0,
            pad_height_divisor,
            pad_width_divisor,
            position,
            border_mode,
            value,
            mask_value,
            always_apply,
            p,
        )
        self.added_height = added_height
        self.added_width = added_width

    def update_params(self, params: Dict[str, Any], **kwargs: Any) -> Dict[str, Any]:
        params = super().update_params(params, **kwargs)
        rows = params["rows"]
        cols = params["cols"]
        self.min_height = rows + self.added_height
        self.min_width = cols + self.added_width
        return super().update_params(params, **kwargs)

In [9]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

bbox_params = A.BboxParams(
    format="coco", min_area=0, min_visibility=0.2, label_fields=["class_labels"]
)

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=bbox_params,
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

# image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# annotations_path = "../data/annotations_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.json"
image_path = "../data/images_full/2023_122000_484000_RGB_hrl.tif"
annotations_path = "../data/annotations_full/3_all_annotations.json"

t = time()

image = cv2.imread(image_path)
bboxes, labels = get_bounding_boxes(annotations_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 10

colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

t = time()

for i in range(number_tests):
    transformed_spatial = transform_spatial(
        image=image, bboxes=bboxes, class_labels=labels
    )
    transformed_spatial_image = transformed_spatial["image"]
    transformed_bboxes = transformed_spatial["bboxes"]
    transformed_class_labels = transformed_spatial["class_labels"]
    transformed = transform_pixel(image=transformed_spatial_image)
    transformed_image = transformed["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    bboxes_image = create_bboxes_image(
        transformed_image,
        transformed_bboxes,
        labels=transformed_class_labels,
        colors_dict=colors,
        scores=np.random.rand((len(bboxes))),
    )

    output_name = f"Augmentation_test_{i}_bboxes.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, bboxes_image)

print(f"Time elapsed: {round(time() - t, 3)}")

Time elapsed: 2.006
Time elapsed: 1.43


In [10]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
    ],
)

transform_2 = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        A.Normalize(p=1.0),
    ],
)

image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# image_path = "../data/flat-design-abstract-outline-background_23-2150616456.jpg"

t = time()

image = cv2.imread(image_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 1

t = time()

for i in range(number_tests):
    r = random.random()
    random.seed(r)
    transformed_image = transform(image=image)["image"]
    random.seed(r)
    transformed_image_2 = transform_2(image=image)["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    output_name = f"Augmentation_test_{i}_2.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image_2)

print(f"Time elapsed: {round(time() - t, 3)}")

Time elapsed: 0.012
Time elapsed: 0.085
